In [2]:
import typer 
from typing import Optional,List 
from phi.assistant import Assistant
from phi.storage.assistant.postgres import PgAssistantStorage
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.pgvector import PgVector2
import os 
from dotenv import load_dotenv

In [ ]:
os.environ["GROQ_API_KEY"] = ""

In [ ]:
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

In [ ]:
knowledge_base = PDFUrlKnowledgeBase(
    urls = ["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db = PgVector2(collection="recipes",db_url=db_url)
)

In [ ]:
knowledge_base.load()

In [ ]:
storage = PgAssistantStorage(table_name="pdf_assistant",db_url=db_url)



In [ ]:
def pdf_assistant(new=False,user="user"):
    run_id = Optional[str] = None
    if not new:
        existing_run_ids : List[str] = storage.get_all_run_ids(user)
        if len(existing_run_ids)>0:
            run_id = existing_run_ids[0]

    assistant = Assistant(
        run_id=run_id,
        user_id= user,
        knowledge_base=knowledge_base,
        storage=storage,
        show_tool_calls=True,
        search_knowledge=True,
        read_chat_history=True

    )
    if run_id is None:
        run_id = assistant.run_id
        print(f"Started_run:{run_id}")
    else:
        print(f"Continuing_run:{run_id}")

    assistant.cli_app(markdown=True)



In [ ]:
typer.run(pdf_assistant)